In [1]:
import os
import sys
import pandas as pd
import ast
import numpy as np
os.chdir('../')


In [3]:
meta_data_path = "../data/metadata_subset.csv"
reviews_data_path = "../data/reviews_subset.csv"
meta_data = pd.read_csv(meta_data_path)
review_data = pd.read_csv(reviews_data_path)
meta_data["asin"] = meta_data["asin"].astype(str)
review_data["asin"] = review_data["asin"].astype(str)

In [4]:
meta_data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"['Software', 'Education &amp; Reference']",NaN,"['Slides with Video, Teaching Public Speaking ...",NaN,Instructor's Resource CD-ROM for The Art of Sp...,[],NaN,McGraw Hill,[],"18,178 in Software (",[],Software,NaN,</div>,$8.00,007742817X,NaN
1,"['Software', 'Education &amp; Reference']",NaN,"[""Contains a guided tour of the program, Plann...",NaN,Magruder's American Government Resource Pro CD...,[],NaN,Magruder's,[],"19,702 in Software (",['0130679550'],Software,NaN,</div>,NaN,0130438480,NaN
2,"['Software', 'Education & Reference', 'Test Pr...",NaN,[],NaN,Prentice Hall Test Manager a Comprehensive Sui...,[],NaN,prentice hall,[],"54,036 in Software (",[],Software,NaN,</div>,NaN,0130852414,NaN
3,"['Software', 'Education &amp; Reference', 'Tes...",NaN,"['Windos 95, 98, NT4, 200, XP\nMac OS 9.1-9.2 ...",NaN,Magruder's American Government Itext Interacti...,[],NaN,Magruder's,['Interactive Learning Tools-Bring Content to ...,"52,031 in Software (",[],Software,NaN,</div>,NaN,0131817949,NaN
4,"['Software', 'Design &amp; Illustration', 'CAD']",NaN,"['2.5 Floppy', '', '']",NaN,AUTOCAD The Student Edition Release 10 (1982-89),[],NaN,Autodesk,[],"30,901 in Software (",[],Software,NaN,</div>,NaN,0201656302,NaN


In [4]:
review_data.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,77613252,Michelle W,The materials arrived early and were in excell...,Material Great,NaN
1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,77613252,Rosalind White Ames,I am really enjoying this book with the worksh...,Health,NaN
2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,77613252,Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,7.0
3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,77613252,Lucy,This book was missing pages!!! Important pages...,missing pages!!,3.0
4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,77775473,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,NaN


### Cleaning meta_data

In [5]:
meta_data['price'] = meta_data.price.str.replace('$','')
meta_data['price'] = meta_data.price.str[:5]
meta_data['price'] = meta_data['price'].str.extract('([0-9]+.[0-9]*)')
meta_data['price'] = meta_data['price'].str.strip()
meta_data['price'] = meta_data['price'].str.replace(',','')
meta_data['price'] = meta_data['price'].astype(float)
meta_data = meta_data[meta_data.price.notnull()]
meta_data['software_category'] = meta_data['category'].apply(lambda x: ast.literal_eval(x)[1])

In [5]:
meta_data.columns.tolist()

['category',
 'tech1',
 'description',
 'fit',
 'title',
 'also_buy',
 'tech2',
 'brand',
 'feature',
 'rank',
 'also_view',
 'main_cat',
 'similar_item',
 'date',
 'price',
 'asin',
 'details',
 'software_category']

### Generating other features

In [6]:
Min_license_fee = meta_data.groupby('software_category').agg(Min_Licensing_Fee=('price', np.min)).reset_index()
meta_data = meta_data.merge(Min_license_fee, on='software_category', how='inner')
meta_data['Licensing_Fee'] = meta_data['Min_Licensing_Fee'] * 0.8
meta_data.drop(columns=['Min_Licensing_Fee'],inplace=True)
meta_data['Implemention_cost'] = meta_data.apply(lambda row: row['price']*0.5, axis=1)
meta_data['Maintenance_cost'] = meta_data.apply(lambda row: row['price']*0.1, axis=1)

In [7]:
meta_data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,main_cat,similar_item,date,price,asin,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost
0,"['Software', 'Education &amp; Reference']",NaN,"['Slides with Video, Teaching Public Speaking ...",NaN,Instructor's Resource CD-ROM for The Art of Sp...,[],NaN,McGraw Hill,[],"18,178 in Software (",...,Software,NaN,</div>,8.00,007742817X,NaN,Education &amp; Reference,5.592,4.000,0.800
1,"['Software', 'Education &amp; Reference']",NaN,"[""Over 50 Activities Developing Essential 1st ...",NaN,SchoolHouse Rock! 1st &amp; 2nd Grade [Win Mac...,[],NaN,Scholastic,['SchoolHouse Rock! 1st &amp; 2nd Grade [Win M...,"12,202 in Software (",...,Software,NaN,</div>,31.47,043983595X,NaN,Education &amp; Reference,5.592,15.735,3.147
2,"['Software', 'Education &amp; Reference', 'Lan...",NaN,['Learn Japanese with the long-awaited sequel ...,NaN,Human Japanese Intermediate,['B00NQI1XAE'],NaN,Brak Software,"['More than 40 illuminating chapters, presente...","6,605 in Software (",...,Software,NaN,</div>,9.98,061591067X,NaN,Education &amp; Reference,5.592,4.990,0.998
3,"['Software', 'Education &amp; Reference']",NaN,['Makes teching and learning mathematics easie...,NaN,Scientific Notebook Version 4.0,[],NaN,MacKichan,['Mathematical word processing plus the easies...,"60,352 in Software (",...,Software,NaN,</div>,74.50,0971562431,NaN,Education &amp; Reference,5.592,37.250,7.450
4,"['Software', 'Education &amp; Reference', 'Lan...",NaN,['The ASL Tales and Games for Kids CD series f...,NaN,ASL Tales and Games for Kids - Leash Lane,[],NaN,IDRT,['The ASL Tales and Games for Kids CD series f...,"14,297 in Software (",...,Software,NaN,</div>,18.00,0981148603,NaN,Education &amp; Reference,5.592,9.000,1.800
